In [14]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_04/rois'

# 01 - Begin Analysis

Enter the path to the dictionary generated by Notebook 01

In [15]:
dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_01/rois/tmp/dataset_dict.json'

Prepare the Files

In [16]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
data_loader = DataLoader(dict_path)

Generate Overlap Maps

In [ ]:
from calvin_utils.ccm_utils.overlap_map import OverlapMap
overlapmapper = OverlapMap(data_loader=data_loader,
            mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii',
            out_dir=out_dir,
            map_type='ROI',      # 'ROI', 't', 'r', 'rfz' or 'custom'
            manual_threshold=None,
            step_size=5,
            verbose=False
        )
overlap_map_dict, stepwise_map_dict = overlapmapper.run()

That's all there is to it

-Calvin